In [71]:
# Multiple Participants -- Approach 2: Model Template
# Train & Test on Distinct Groups of Participants
# Approach 2: Use Small % of Test Participant's Data (Chronological Split)

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [73]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = True

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = False

# Number of Estimators for Bagging
num_estimators = 10

# Sample Percentage for Bagging
sample_percentage = 0.75

# Percentage of Test Participant Data to Use
test_percentage_to_use = 0.3

# File Name for Model Results
output_file = 'multi2_kf_knn_results.csv'

# 1 -- Load Data

In [74]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [75]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [76]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [77]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [78]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [79]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [80]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [81]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [82]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [83]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [84]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [85]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [86]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [87]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [88]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    val_data = []
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            val_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    val_data = pd.concat(val_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training & Validation Data 
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)

    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])        
    X_val = val_data.drop(columns=['engagement', 'session_num', 'participant'])  
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    y_train = train_data['engagement']
    y_val = val_data['engagement']
    y_test = test_data['engagement']

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_val[c] = (X_val[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_val[c] = (X_val[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_val[c] = X_val[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_val[c] = X_val[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_val, y_val, X_test, y_test

In [89]:
# MODEL HERE
# Multiple Participants Approach 2: 
# Approach 2: Use Small % of Test Participant's Data (Chronological Split)

# Inputs: X_train, y_train, X_val, y_val, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from sklearn import neighbors

def model(X_train, y_train, X_val, y_val, X_test, y_test):
    # Model Here: must create pred & scores arrays
    #clf = BaggingClassifier(n_estimators=num_estimators, max_samples=sample_percentage)
    
    X_train_new = X_train.append(X_val)
    y_train_new = y_train.append(y_val)
    
    idx = np.random.permutation(len(X_train_new))
    X_train_new = X_train_new.iloc[idx]
    y_train_new = y_train_new.iloc[idx]
    X_train_new = X_train_new.reset_index(drop=True)
    y_train_new = y_train_new.reset_index(drop=True)

    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train_new, y_train_new)
    scores = clf.predict_proba(X_test.values)[:,1]
    pred = [round(value) for value in scores]   
        
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    try:
        auc = roc_auc_score(y_test, scores)
    except:
        auc = np.nan
    results['auc'] = auc

    precision = metrics.precision_score(y_test, pred, average=None)
    recall = metrics.recall_score(y_test, pred, average=None)
    f1 = metrics.f1_score(y_test, pred, average=None)

    results['precision_0'], results['precision_1'] = precision[0], precision[1]
    results['recall_0'], results['recall_1'] = recall[0], recall[1]
    results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [90]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_val, y_val, X_test, y_test = split(train_part, test_part, test_percentage_to_use)
        #X_train = X_train.reset_index(drop=True)
        #y_train = y_train.reset_index(drop=True)
        #X_val = X_val.reset_index(drop=True)
        #y_val = y_val.reset_index(drop=True)

        # Evaluate on Test  
        results = model(X_train, y_train, X_val, y_val, X_test, y_test)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
{'recall_0': 0.5347671747755273, 'accuracy': 0.7115413033840697, 'precision_1': 0.7103484139365575, 'precision_0': 0.7140998234036621, 'recall_1': 0.8419971234846928, 'auc': 0.7245285640710113, 'f1_0': 0.6115577489453157, 'f1_1': 0.7705904475366679}

Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
{'recall_0': 0.45606595995288574, 'accuracy': 0.6983931559175427, 'precision_1': 0.701961965407452, 'precision_0': 0.6885595732068761, 'recall_1': 0.8613134832647028, 'auc': 0.6997384945932736, 'f1_0': 0.5487009919697685, 'f1_1': 0.773516025033188}

Train: [9] 	Test: [5, 7, 11, 12, 17, 18]
{'recall_0': 0.5328374455732946, 'accuracy': 0.7305134000645787, 'precision_1': 0.7648401826484018, 'precision_0': 0.6476295479603087, 'recall_1': 0.8397673719041412, 'auc': 0.7370949549996892, 'f1_0': 0.5846521349656613, 'f1_1': 0.800554413802992}

Train: [11] 	Test: [5, 7, 9, 12, 17, 18]
{'recall_0': 0.5329744279946165, 'accuracy': 0.698688822536674, 'precision_1': 0.69

{'recall_0': 0.45257021727609964, 'accuracy': 0.7087212616378491, 'precision_1': 0.7357380404195446, 'precision_0': 0.6307237813884786, 'recall_1': 0.8518957345971564, 'auc': 0.7074922608261314, 'f1_0': 0.5269978401727862, 'f1_1': 0.7895676046671243}

Train: [5, 7, 11] 	Test: [9, 12, 17, 18]
{'recall_0': 0.5046474035158618, 'accuracy': 0.6843112855365177, 'precision_1': 0.6548641419118683, 'precision_0': 0.74686004784689, 'recall_1': 0.8460349217897417, 'auc': 0.7163330430191518, 'f1_0': 0.6023152055950802, 'f1_1': 0.738274740099992}

Train: [5, 7, 12] 	Test: [9, 11, 17, 18]
{'recall_0': 0.44624447717231225, 'accuracy': 0.6851964007613774, 'precision_1': 0.6878002490955459, 'precision_0': 0.6781774580335731, 'recall_1': 0.8520940484937546, 'auc': 0.6876916091356818, 'f1_0': 0.5382907175940613, 'f1_1': 0.7611827639394834}

Train: [5, 7, 17] 	Test: [9, 11, 12, 18]
{'recall_0': 0.48208370972598613, 'accuracy': 0.6932382263379635, 'precision_1': 0.6960414703110274, 'precision_0': 0.6864370

{'recall_0': 0.5432951249827372, 'accuracy': 0.7221603293775732, 'precision_1': 0.7684497969472063, 'precision_0': 0.6182618261826183, 'recall_1': 0.8187854371829305, 'auc': 0.7274086960707478, 'f1_0': 0.5783593060864453, 'f1_1': 0.792819475547208}

Train: [9, 17, 18] 	Test: [5, 7, 11, 12]
{'recall_0': 0.6054819433619122, 'accuracy': 0.7387219793507581, 'precision_1': 0.7870266479663394, 'precision_0': 0.6436067384700359, 'recall_1': 0.8130252100840336, 'auc': 0.75701916614739, 'f1_0': 0.623962516733601, 'f1_1': 0.7998147092360318}

Train: [11, 12, 17] 	Test: [5, 7, 9, 18]
{'recall_0': 0.5918411630682434, 'accuracy': 0.7129152284036401, 'precision_1': 0.7352568613652357, 'precision_0': 0.6737063109793751, 'recall_1': 0.79816653934301, 'auc': 0.7333192485768607, 'f1_0': 0.6301259096684763, 'f1_1': 0.7654212454212453}

Train: [11, 12, 18] 	Test: [5, 7, 9, 17]
{'recall_0': 0.5844020435825252, 'accuracy': 0.7041538536632986, 'precision_1': 0.6887639572109003, 'precision_0': 0.7298177083333

{'recall_0': 0.3232089033155576, 'accuracy': 0.707615511048883, 'precision_1': 0.7780059320100388, 'precision_0': 0.42525930445393534, 'recall_1': 0.8444774640911342, 'auc': 0.6417741747854001, 'f1_0': 0.3672770385983401, 'f1_1': 0.8098800617503858}

Train: [7, 9, 12, 18] 	Test: [5, 11, 17]
{'recall_0': 0.4038830808619586, 'accuracy': 0.6991168617785993, 'precision_1': 0.7485827409340412, 'precision_0': 0.5417859187178019, 'recall_1': 0.838608870967742, 'auc': 0.6740127764165812, 'f1_0': 0.4627796112944628, 'f1_1': 0.7910426472685779}

Train: [7, 9, 17, 18] 	Test: [5, 11, 12]
{'recall_0': 0.5, 'accuracy': 0.7264614588722194, 'precision_1': 0.7710521630763753, 'precision_0': 0.6080378250591016, 'recall_1': 0.8393410852713178, 'auc': 0.7193225418038024, 'f1_0': 0.548751866865799, 'f1_1': 0.8037487241347313}

Train: [7, 11, 12, 17] 	Test: [5, 9, 18]
{'recall_0': 0.5060783430886988, 'accuracy': 0.7017026246235171, 'precision_1': 0.7096603440670489, 'precision_0': 0.6834211593027969, 'recal

{'recall_0': 0.6299163827810468, 'accuracy': 0.7628781684382666, 'precision_1': 0.7900562192550948, 'precision_0': 0.7089578250261415, 'recall_1': 0.8433983495873969, 'auc': 0.7881124721409525, 'f1_0': 0.6671039685142669, 'f1_1': 0.815856313497823}

Train: [7, 9, 12, 17, 18] 	Test: [5, 11]
{'recall_0': 0.3475502291152626, 'accuracy': 0.7071315960307892, 'precision_1': 0.7819787985865725, 'precision_0': 0.4300043610989969, 'recall_1': 0.83551472438963, 'auc': 0.6416417089588065, 'f1_0': 0.38440545808966864, 'f1_1': 0.8078607933803846}

Train: [7, 11, 12, 17, 18] 	Test: [5, 9]
{'recall_0': 0.5515712357817621, 'accuracy': 0.6938352941176471, 'precision_1': 0.6597923065672079, 'precision_0': 0.7552798310454065, 'recall_1': 0.8295329165134241, 'auc': 0.7157306618457198, 'f1_0': 0.6375487465181059, 'f1_1': 0.734989816700611}

Train: [9, 11, 12, 17, 18] 	Test: [5, 7]
{'recall_0': 0.7059838895281934, 'accuracy': 0.758419036095987, 'precision_1': 0.832183908045977, 'precision_0': 0.641065830721

In [91]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [92]:
results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.711541,0.724529,0.611558,0.770590,0.714100,0.710348,0.534767,0.841997,"7, 9, 11, 12, 17, 18",5
1,0.698393,0.699738,0.548701,0.773516,0.688560,0.701962,0.456066,0.861313,"5, 9, 11, 12, 17, 18",7
2,0.730513,0.737095,0.584652,0.800554,0.647630,0.764840,0.532837,0.839767,"5, 7, 11, 12, 17, 18",9
3,0.698689,0.717026,0.605607,0.756223,0.701161,0.697472,0.532974,0.825783,"5, 7, 9, 12, 17, 18",11
4,0.710056,0.718641,0.585727,0.776986,0.667883,0.728730,0.521568,0.832087,"5, 7, 9, 11, 17, 18",12


In [93]:
results.to_csv(output_file, index=False)